# Task 0

In [1]:
pip install geopandas altair pandas jupyter

Note: you may need to restart the kernel to use updated packages.


# Task 1:

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
from shapely.geometry import Point

df = pd.read_csv('/Users/manasadattakandimalla/Downloads/CS424-Lab 3/Taxi_Trips.csv')
geometry = [Point(xy) for xy in zip(df['Pickup Centroid Longitude'], df['Pickup Centroid Latitude'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs=4326).sample(1000)
gdf = gdf.rename(columns={"Trip Seconds": "Trip_Seconds", "Trip Miles": "Trip_Miles"})

In [3]:
gdf.head()

,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip_Seconds,Trip_Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,...,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location,geometry
63234,8d0da516d29134a1fe38a3ff38e7bffcb1c73fb4,f5b5de5c0c42601f6878d82eeb1984d06c977aab60cb0d...,07/07/2022 11:30:00 AM,07/07/2022 11:45:00 AM,1200.0,0.30,NaN,NaN,34.0,37.0,...,18.50,Unknown,Taxi Affiliation Services,41.842076,-87.633973,POINT (-87.6339734222 41.8420761168),41.809084,-87.632425,POINT (-87.6324245245 41.8090844303),POINT (-87.63397 41.84208)
66108,6b64bb409a807416fdeb55f48dffedf0e82ba53c,e637e1afe0dab0e2d04c635335602caf1045d43e3741c2...,07/07/2022 02:00:00 PM,07/07/2022 02:00:00 PM,480.0,0.00,1.703184e+10,1.703132e+10,32.0,32.0,...,6.50,Cash,Taxi Affiliation Services,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.884987,-87.620993,POINT (-87.6209929134 41.8849871918),POINT (-87.63275 41.88099)
88101,9405c3f8f04e16f4d45e1cf7d660a187f68ac091,791d74df896226a452a8e223e2ec9fa0df7d80bb7ca180...,07/08/2022 05:15:00 PM,07/08/2022 06:15:00 PM,3600.0,16.80,1.703198e+10,1.703183e+10,76.0,28.0,...,56.50,Credit Card,Taxi Affiliation Services,41.979071,-87.903040,POINT (-87.9030396611 41.9790708201),41.885281,-87.657233,POINT (-87.6572331997 41.8852813201),POINT (-87.90304 41.97907)
19184,8ea27993206011b779446035822e877741cc9d12,b515261b90c81a5966755dbbb08f7cf0ff3928a4c6efb7...,07/04/2022 04:00:00 PM,07/04/2022 04:15:00 PM,1500.0,13.20,1.703198e+10,1.703108e+10,56.0,8.0,...,46.00,Credit Card,"Taxicab Insurance Agency, LLC",41.785999,-87.750934,POINT (-87.7509342894 41.785998518),41.895033,-87.619711,POINT (-87.6197106717 41.8950334495),POINT (-87.75093 41.786)
41466,2fa0e111fa9bf33628b7a5130f94b1a44a8b9b13,3beee2ac6f2972630c37deb940e1b66e774e22858b4acc...,07/06/2022 02:00:00 AM,07/06/2022 02:30:00 AM,2059.0,9.65,NaN,NaN,NaN,25.0,...,28.25,Cash,Sun Taxi,NaN,NaN,NaN,41.890609,-87.756047,POINT (-87.7560467111 41.8906088526),POINT EMPTY


# Task 2:

In [4]:
chicago = gpd.read_file('/Users/manasadattakandimalla/Downloads/CS424-Lab 3/chicago.geojson')

In [5]:
joined = gpd.sjoin(gdf, chicago, predicate='within')
joined = joined.groupby('zip').mean(numeric_only=True)
joined = joined.filter(['Fare'])

In [6]:
merged = chicago.merge(joined, on='zip')

# Task 3:

In [7]:
brush = alt.selection_interval()

matrix = alt.Chart(gdf).mark_circle().add_params(brush).encode(
    alt.X(alt.repeat("column"), type='quantitative'),
    alt.Y(alt.repeat("row"), type='quantitative'),
    color=alt.condition(brush, 'Payment Type:N', alt.value('grey')),
    opacity=alt.condition(brush, alt.value(0.8), alt.value(0.1))
).properties(
    width=150,
    height=150
).repeat(
    row=['Fare', 'Trip_Miles', 'Trip_Seconds'],
    column=['Trip_Seconds', 'Trip_Miles', 'Fare']
)

matrix

alt.RepeatChart(...)

In [8]:
scatter = alt.Chart(gdf).mark_circle().encode(
    x=alt.Y('Pickup Centroid Longitude',scale=alt.Scale(domain=[-88.0, -87.5])),
    y=alt.Y('Pickup Centroid Latitude',scale=alt.Scale(domain=[41.6, 42.1])),
    color='Fare',
    opacity=alt.condition(brush, alt.value(1), alt.value(0))
)

In [9]:
(scatter & matrix).add_params(brush)

alt.VConcatChart(...)

# Task 4 (Bubble Plot)

In [11]:
import altair as alt

brush = alt.selection_interval()

# Bubble plot for spatial attributes (Pickup locations)
bubble_scatter = alt.Chart(gdf).mark_circle().encode(
    x=alt.X('Pickup Centroid Longitude', scale=alt.Scale(domain=[-88.0, -87.5])),
    y=alt.Y('Pickup Centroid Latitude', scale=alt.Scale(domain=[41.6, 42.1])),
    size=alt.Size('Fare', scale=alt.Scale(range=[10, 200]), legend=None),
    color=alt.Color('Fare', scale=alt.Scale(scheme='purples')),
    opacity=alt.condition(brush, alt.value(0.8), alt.value(0.1))
).add_params(brush).properties(
    width=400,
    height=400
)

# Bubble matrix for linking
bubble_matrix = alt.Chart(gdf).mark_circle().encode(
    alt.X(alt.repeat("column"), type='quantitative'),
    alt.Y(alt.repeat("row"), type='quantitative'),
    size=alt.Size('Fare', scale=alt.Scale(range=[10, 200]), legend=None),
    color=alt.condition(brush, alt.Color('Payment Type:N', scale=alt.Scale(scheme='purples')), alt.value('grey')),
    opacity=alt.condition(brush, alt.value(0.8), alt.value(0.1))
).properties(
    width=150,
    height=150
).repeat(
    row=['Fare', 'Trip_Miles', 'Trip_Seconds'],
    column=['Trip_Seconds', 'Trip_Miles', 'Fare']
).add_params(brush)

# Combine both charts
(bubble_scatter & bubble_matrix)

alt.VConcatChart(...)